In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import sys
sys.path.append('/content/gdrive/My Drive/MNIST_CayleyNet')

In [3]:
!pip install scipy==1.0.0

In [4]:
import tensorflow as tf
import time, shutil
import numpy as np
import os, collections, sklearn
import joblib
import cv2
import graph, coarsening
import scipy.sparse as sp

import matplotlib.pyplot as plt
from scipy.misc import imresize
%matplotlib inline

In [0]:
#Definition of some flags useful later in the code

flags = tf.app.flags
FLAGS = flags.FLAGS

# Graphs.
flags.DEFINE_string('f', '', 'kernel')
flags.DEFINE_integer('number_edges', 4, 'Graph: minimum number of edges per vertex.')
flags.DEFINE_string('metric', 'euclidean', 'Graph: similarity measure (between features).')
flags.DEFINE_bool('normalized_laplacian', True, 'Graph Laplacian: normalized.')


# Directories.
flags.DEFINE_string('dir_data', 'data_mnist', 'Directory to store data.')

In [6]:
#Here we proceed at computing the original grid where the images live and the various coarsening that are applied
#for each level
new_dim = 24
pool_step = 2
def laplacian2D(N, h):
    diag=np.ones([N*N])
    mat=sp.spdiags([-diag,2*diag,-diag],[-1,0,1],N,N)
    mat = mat.tocsr()
    mat[0, N-1] = -1.0
    mat[N-1, 0] = -1.0
    I=sp.eye(N)
    return (sp.kron(I,mat,format='csr')+sp.kron(mat,I,format='csr'))/(h**2)
L = []
L.append(laplacian2D(new_dim, 2.0*(56/new_dim)))
L.append(laplacian2D(new_dim//pool_step, 2.0*pool_step*(56/new_dim)))
L.append(laplacian2D(new_dim//pool_step**2, 2.0*pool_step**2*(56/new_dim)))
L_r = []
L_i = []
h = 2.0
for c_L in L:
    c_L = c_L.toarray()
    c_L = (h*c_L - 1j*np.eye(c_L.shape[0])).dot(np.linalg.inv(h*c_L + 1j*np.eye(c_L.shape[0])))
    L_r.append(c_L.real)
    L_i.append(c_L.imag)

/usr/local/lib/python3.6/dist-packages/scipy/sparse/compressed.py:742: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [0]:
#Normalize Laplacian
L_norm = []
for k in range(len(L)):
    L_norm.append(L[k])

In [8]:
#loading of MNIST dataset

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(FLAGS.dir_data, one_hot=False)

train_data = mnist.train.images.astype(np.float32)
val_data = mnist.validation.images.astype(np.float32) #the first 5K samples of the training dataset 
                                                      #are used for validation
test_data = mnist.test.images.astype(np.float32)
train_labels = mnist.train.labels
val_labels = mnist.validation.labels
test_labels = mnist.test.labels
new_train_data = np.zeros((train_data.shape[0], new_dim*new_dim))
new_val_data = np.zeros((val_data.shape[0], new_dim*new_dim))
new_test_data = np.zeros((test_data.shape[0], new_dim*new_dim))
for i in range(train_data.shape[0]):
    im = train_data[i,:].reshape(28,28)
    im = cv2.resize(im, (new_dim,new_dim), interpolation = cv2.INTER_LINEAR)
    new_train_data[i,:] = im.reshape(1,-1)
for i in range(val_data.shape[0]):
    im = val_data[i,:].reshape(28,28)
    im = cv2.resize(im, (new_dim,new_dim), interpolation = cv2.INTER_LINEAR)
    new_val_data[i,:] = im.reshape(1,-1)
for i in range(test_data.shape[0]):
    im = test_data[i,:].reshape(28,28)
    im = cv2.resize(im, (new_dim,new_dim), interpolation = cv2.INTER_LINEAR)
    new_test_data[i,:] = im.reshape(1,-1)

t_start = time.time()
train_data = new_train_data
val_data = new_val_data
test_data = new_test_data
print('Execution time: {:.2f}s'.format(time.time() - t_start))

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data_mnist/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data_mnist/train-labels-idx1-ubyte.gz
Extracting data_mnist/t10k-images-idx3-ubyte.gz
Extracting data_mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Execution time: 0.00s


In [0]:
class CayleyNet:
    """
    The neural network model.
    """
    
    #Helper functions used for constructing the model
    def _weight_variable(self, shape, regularization=True, name=""): 
        """Initializer for the weights"""
        
        initial = tf.truncated_normal_initializer(0, 0.1)
        var = tf.get_variable('weights'+name, shape, tf.float32, initializer=initial)
        if regularization: #append the loss of the current variable to the regularization term 
            self.regularizers.append(tf.nn.l2_loss(var))
        return var
    
    def _bias_variable(self, shape, regularization=True):
        """Initializer for the bias"""
        
        initial = tf.constant_initializer(0.1)
        var = tf.get_variable('bias', shape, tf.float32, initializer=initial)
        if regularization:
            self.regularizers.append(tf.nn.l2_loss(var))
        return var
    
    def _h_variable(self, shape, regularization=False, name=''):
        """Initializer for the zoom parameter h"""
        
        initial = tf.random_uniform_initializer()
        var = tf.get_variable('h'+name, shape, tf.float32, initializer=initial)
        if regularization:
            self.regularizers.append(tf.nn.l2_loss(var))
        return var

    def frobenius_norm(self, tensor): 
        """Computes the frobenius norm for a given laplacian"""
        
        square_tensor = tf.square(tensor)
        tensor_sum = tf.reduce_sum(square_tensor)
        frobenius_norm = tf.sqrt(tensor_sum)
        return frobenius_norm
    
    def compute_sparse_D_inv_indices(self, M):
        """Computes the indices required for constructing a sparse version of D^-1."""
        
        idx_main_diag = np.tile(np.expand_dims(np.arange(0, 2*M),1), [1, 2])
        idx_diag_ur = np.concatenate([np.expand_dims(np.arange(0, M),1), np.expand_dims(np.arange(0, M)+M,1)], 1)
        idx_diag_ll = np.concatenate([np.expand_dims(np.arange(0, M)+M,1), np.expand_dims(np.arange(0, M),1)], 1)
        idx = np.concatenate([idx_main_diag, idx_diag_ur, idx_diag_ll], 0)
        return idx  
    
    def compute_sparse_R_indices(self, L_off_diag, M):
        """Computes the indices required for constructing a sparse version of R."""
        
        idx_L = np.asarray(np.where(L_off_diag)).T
        idx_L_sh = idx_L + np.expand_dims(np.asarray([M,M]),0)
        idx = np.concatenate([idx_L, idx_L_sh])
        return idx
    
    def compute_sparse_numerator_projection_indices(self, L, M):
        """Computes the indices required for constructing the numerator projection sparse matrix."""
        
        idx_L = np.asarray(np.where(L)).T
        idx_L_sh = idx_L + np.expand_dims(np.asarray([M,M]),0)
        idx_diag_ur = np.concatenate([np.expand_dims(np.arange(0, M),1), np.expand_dims(np.arange(0, M)+M,1)], 1)
        idx_diag_ll = np.concatenate([np.expand_dims(np.arange(0, M)+M,1), np.expand_dims(np.arange(0, M),1)], 1)
        idx = np.concatenate([idx_L, idx_L_sh, idx_diag_ur, idx_diag_ll])
        return idx
    
    def chebyshevConv(self, x, L, Fout, K): 
        """Applies chebyshev polynomials over the graph (i.e. it makes a spectral convolution)"""
        
        N, M, Fin = x.get_shape()  # N is the number of images
                                  # M the number of vertices in the images
                                  # Fin the number of features
        N, M, Fin = int(N), int(M), int(Fin)
        
        x = tf.cast(x, 'complex64')
        x = tf.transpose(x, [1,2,0])
        x = tf.reshape(x, [M, Fin*N])
        def concat(x, x_):
            x_ = tf.expand_dims(x_, 0)
            return tf.concat([x, x_], 0)
        output = tf.expand_dims(x, 0)
        for i in range(K):
            x = tf.matmul(L, x)
            output = concat(output, x)
  
        x = tf.reshape(output, [K+1, M, Fin, N])
        x = tf.transpose(x, [3,1,2,0]) # shape = N x M x Fin x (K + 1)
        x = tf.reshape(x, [N*M, Fin*(K+1)])  # shape = N*M x Fin*(K + 1)

        real_conv_weights = self._weight_variable([Fin*(K+1), Fout], regularization=False, name='_real')#tf.ones([Fin*(self.n_h*K+1), Fout])#self._weight_variable([Fin*(self.n_h*K+1), Fout], regularization=False, name='_real')
        imag_conv_weights = self._weight_variable([Fin*(K+1), Fout], regularization=False, name='_imag')#tf.ones([Fin*(self.n_h*K+1), Fout])#self._weight_variable([Fin*(self.n_h*K+1), Fout], regularization=False, name='_imag')

        W_pos_exp = tf.complex(real_conv_weights, -imag_conv_weights)
        
        x_pos_exp_filt = tf.matmul(x, W_pos_exp)
        
        x_filt = 2*tf.real(x_pos_exp_filt)
        return tf.reshape(x_filt, [N, M, Fout])  # N x M x Fout 

    def cayleyConv(self, x, L_np, Fout, K): 
        """Applies chebyshev polynomials over the graph."""
        
        M, Fin = x.get_shape()[1:] # M the number of samples in the images, Fin the number of features
        M, Fin = int(M), int(Fin)
        N = tf.shape(x)[0] # N is the number of images
        
        # Applies cayley transform by means of Jacobi method.
        diag_L_np = np.diag(L_np)  # vector containing the diagonal of L
        L_off_diag_np = L_np - np.diag(diag_L_np) # off-diagonal entries of L 
        
        list_x_pos_exp = [tf.cast(tf.expand_dims(x,0), 'complex64')] # 1 x N x M x F
        
        for iii in range(self.n_h):  # for every zoom parameter we want to use (typically one).
            h = self._h_variable([1,1], regularization=False, name='_h%f' % iii)
            self.list_h.append(h)
            
            # Computes matrices required by Jacobi (https://en.wikipedia.org/wiki/Jacobi_method)
            
            # To make things more efficient we reprent a complex vector of shape M as real vector of shape 2*M
            # where the first M values represent real coefficients while the second M the imaginary ones.
            # All the matrices here defined are computed according to such notation (it allows to use sparse matrices
            # with TF with complex values).
            
            # ************************** COMPUTES numerator projection **************************
            idx = self.compute_sparse_numerator_projection_indices(L_np, M)
            
            vals_L = tf.squeeze(h*L_np[np.where(L_np)])
            vals = tf.concat([vals_L, vals_L, tf.ones([M,]), -tf.ones([M,])], 0)
            
            cayley_op_neg_sp = tf.SparseTensor(idx, vals, [M*2, M*2])
            cayley_op_neg_sp = tf.sparse_reorder(cayley_op_neg_sp)
        
            # ************************** COMPUTES D **************************
            D_real = tf.squeeze(h*diag_L_np)
            D = tf.complex(D_real, tf.ones_like(D_real))
            D_inv = tf.pow(D, -tf.ones_like(D)) # vector of M elements <- diagonal of D^-1
            
            idx = self.compute_sparse_D_inv_indices(M)
            vals = tf.concat([tf.real(D_inv), tf.real(D_inv), -tf.imag(D_inv), tf.imag(D_inv)], 0)
            
            D_inv_ext_sp = tf.SparseTensor(idx, vals, [M*2, M*2])
            D_inv_ext_sp = tf.sparse_reorder(D_inv_ext_sp)
            
            # ************************** COMPUTES R **************************
            idx = self.compute_sparse_R_indices(L_off_diag_np, M)
            
            vals_L = tf.squeeze(h*L_off_diag_np[np.where(L_off_diag_np)])
            vals = tf.concat([vals_L, vals_L], 0)
            
            R_sp = tf.SparseTensor(idx, vals, [M*2, M*2])
            R_sp = tf.sparse_reorder(R_sp)
            
            # Applies Jacobi method
            c_transform = tf.transpose(x, [1,0,2]) # shape = M, N, F
            c_transform = tf.reshape(c_transform, [M, -1]) # shape = M, N*F
            last_sol = tf.concat([c_transform, tf.zeros_like(c_transform)],0)
            for k in range(K):  # for every order of our polynomial
                
                # Jacobi initialization
                b = tf.sparse_tensor_dense_matmul(cayley_op_neg_sp, last_sol) # shape = M, N*F
                a = tf.sparse_tensor_dense_matmul(D_inv_ext_sp, b) # shape = M, N*F
                
                # Jacobi iterations
                cond = lambda i, _: tf.less(i, self.num_jacobi_iter)
                body = lambda i, c_sol: [tf.add(i, 1), a  - tf.sparse_tensor_dense_matmul(D_inv_ext_sp, 
                                                                                          tf.sparse_tensor_dense_matmul(R_sp, c_sol))]
                
                c_sol = tf.while_loop(cond, body, [0, a], parallel_iterations=1, swap_memory=True)
                c_sol = c_sol[-1]
                    
                # Constructs and saves the final complex matrices
                c_sol_complex = tf.complex(c_sol[:M,:], c_sol[M:, :]) #M x N*F
                c_sol_reshaped = tf.reshape(c_sol_complex, [M, -1, Fin])
                c_sol_reshaped = tf.transpose(c_sol_reshaped, [1, 0, 2]) #N x M x F
                list_x_pos_exp.append(tf.expand_dims(c_sol_reshaped,0)) #1 x N x M x Flist_x_pos_exp
                
                last_sol = c_sol
        x_pos_exp = tf.concat(list_x_pos_exp, 0) # shape = n_h*K x N x M x Fin
        x_pos_exp = tf.transpose(x_pos_exp, [1,2,0,3])  #N x M x n_h*K x Fin
        x_pos_exp = tf.reshape(x_pos_exp, [N*M, -1]) #N*M x 2*K*Fin
        
        real_conv_weights = self._weight_variable([Fin*(self.n_h*K+1), Fout], regularization=False, name='_real')#tf.ones([Fin*(self.n_h*K+1), Fout])#self._weight_variable([Fin*(self.n_h*K+1), Fout], regularization=False, name='_real')
        imag_conv_weights = self._weight_variable([Fin*(self.n_h*K+1), Fout], regularization=False, name='_imag')#tf.ones([Fin*(self.n_h*K+1), Fout])#self._weight_variable([Fin*(self.n_h*K+1), Fout], regularization=False, name='_imag')
        
        W_pos_exp = tf.complex(real_conv_weights, -imag_conv_weights)
        
        x_pos_exp_filt = tf.matmul(x_pos_exp, W_pos_exp)
        
        x_filt = 2*tf.real(x_pos_exp_filt)
        return tf.reshape(x_filt, [N, M, Fout])


    def b1relu(self, x): #sums a bias and applies relu
        """Bias and ReLU. One bias per filter."""
        N, M, F = x.get_shape()
        b = self._bias_variable([1, 1, int(F)], regularization=False)
        return tf.nn.relu(x + b) #add the bias to the convolutive layer


    def mpool1(self, x, p): #efficient pooling realized thanks to the reordering of the laplacians we have done a priori
        """Max pooling of size p. Should be a power of 2."""
        if p > 1:
            x = tf.expand_dims(x, 3)  # N x M x F x 1
            x = tf.nn.max_pool(x, ksize=[1,p,1,1], strides=[1,p,1,1], padding='SAME')
            return tf.squeeze(x, [3])  # N x M/p x F
        else:
            return x
    

    def b1relu(self, x): #sums a bias and applies relu
        """Bias and ReLU. One bias per filter."""
        N, M, F = x.get_shape()
        b = self._bias_variable([1, 1, int(F)], regularization=False)
        return tf.nn.relu(x + b) #add the bias to the convolutive layer


    def mpool1(self, x, p): #efficient pooling realized thanks to the reordering of the laplacians we have done a priori
        """Max pooling of size p. Should be a power of 2."""
        if p > 1:
            x = tf.expand_dims(x, 3)  # N x M x F x 1
            x = tf.nn.max_pool(x, ksize=[1,p,1,1], strides=[1,p,1,1], padding='SAME')
            return tf.squeeze(x, [3])  # N x M/p x F
        else:
            return x

    def fc(self, x, Mout, relu=True):
        """Fully connected layer with Mout features."""
        N, Min = x.get_shape()
        W = self._weight_variable([int(Min), Mout], regularization=True)
        b = self._bias_variable([Mout], regularization=True)
        x = tf.matmul(x, W) + b
        return tf.nn.relu(x) if relu else x
      
    def count_no_weights(self):
        total_parameters = 0
        for variable in tf.trainable_variables():
            # shape is an array of tf.Dimension
            shape = variable.get_shape()
            variable_parameters = 1
            for dim in shape:
                variable_parameters *= dim.value
            total_parameters += variable_parameters
        print('#weights in the model: %d' % (total_parameters,))
    
    #function used for extracting the result of our model
    def _inference(self, x, dropout): #definition of the model
        
        # Graph convolutional layers.
        x = tf.expand_dims(x, 2)  # N x M x F=1
        j = 0
        self.list_h = list()
        rec_size = [new_dim, new_dim//pool_step, new_dim//(pool_step**2)]
        for i in range(len(self.p)):
            with tf.variable_scope('cgconv{}'.format(i+1)):
                with tf.name_scope('filter'):
                    x = self.chebyshevConv(x, self.L_np[i], self.F[i], self.K[i])
                    if (i==0):
                        self.debug = x
                with tf.name_scope('bias_relu'):
                    x = self.b1relu(tf.cast(tf.real(x), 'float32'))
                with tf.name_scope('pooling'):
                    N, M, F = x.get_shape()
                    x = tf.reshape(x, [N, rec_size[i],rec_size[i], F])
                    x = tf.contrib.layers.instance_norm(x)
                    x = tf.nn.max_pool(x, ksize=[1,self.p[i],self.p[i],1], strides=[1,self.p[i],self.p[i],1], padding='SAME')
                    x = tf.reshape(x, [N, -1, F])
            j += int(np.log2(self.p[i])) if self.p[i] > 1 else 0
        
        # Fully connected hidden layers.
        _, M, F = x.get_shape()
        x = tf.reshape(x, [-1, int(M*F)])  # N x M
        if self.M:
            for i,M in enumerate(self.M[:-1]): #apply a fully connected layer for each layer defined in M
                                              #(we discard the last value in M since it contains the number of classes we have
                                              #to predict)
                with tf.variable_scope('fc{}'.format(i+1)):
                    x = self.fc(x, M)
                    x = tf.nn.dropout(x, dropout)
            
            # Logits linear layer, i.e. softmax without normalization.
            with tf.variable_scope('logits'):
                x = self.fc(x, self.M[-1], relu=False)
        return x
    
    def __init__(self, p, K, F, M, M_0, batch_size, num_jacobi_iter, L_r, L_i,
                 decay_steps, decay_rate, learning_rate=1e-4, momentum=0.9, regularization=5e-4, clip_norm=1e1,
                 idx_gpu = '/gpu:0'):
        self.regularizers = list() #list of regularization l2 loss for multiple variables
        self.n_h = 1
        self.num_jacobi_iter = num_jacobi_iter
        self.p = p #dimensions of the pooling layers
        self.K = K #List of polynomial orders, i.e. filter sizes or number of hops
        self.F = F #Number of features of convolutional layers
        
        self.M = M #Number of neurons in fully connected layers
        
        self.M_0 = M_0 #number of elements in the first graph 
        
        self.batch_size = batch_size
        
        #definition of some learning parameters
        self.decay_steps = decay_steps
        self.decay_rate = decay_rate
        self.learning_rate = learning_rate
        self.regularization = regularization
        
        with tf.Graph().as_default() as g:
                self.graph = g
                tf.set_random_seed(0)
                with tf.device(idx_gpu):
                        #definition of placeholders
                        self.L_np = [tf.complex(tf.convert_to_tensor(c_L_r), tf.convert_to_tensor(c_L_i)) for c_L_r, c_L_i in zip(L_r, L_i)]
                        self.L_np = [tf.cast(c_L, 'complex64') for c_L in self.L_np]
                        self.ph_data = tf.placeholder(tf.float32, (self.batch_size, M_0), 'data')
                        self.ph_labels = tf.placeholder(tf.int32, (self.batch_size), 'labels')
                        self.ph_dropout = tf.placeholder(tf.float32, (), 'dropout')
                    
                        #Model construction
                        self.logits = self._inference(self.ph_data, self.ph_dropout)
                        
                        #Definition of the loss function
                        with tf.name_scope('loss'):
                            self.cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=self.logits, labels=self.ph_labels)
                            self.cross_entropy = tf.reduce_mean(self.cross_entropy)
                        with tf.name_scope('regularization'):
                            if self.M:
                                self.regularization *= tf.add_n(self.regularizers)
                        self.loss = self.cross_entropy + self.regularization
                        # Create a session for running Ops on the Graph.
                        config = tf.ConfigProto(allow_soft_placement = True)
                        config.gpu_options.allow_growth = True
                        self.session = tf.Session(config=config)
                        #Solver Definition
                        with tf.name_scope('training'):
                            # Learning rate.
                            global_step = tf.Variable(0, name='global_step', trainable=False) #used for counting how many iterations we have done
                            if decay_rate != 1: #applies an exponential decay of the lr wrt the number of iterations done
                                learning_rate = tf.train.exponential_decay(
                                        learning_rate, global_step, decay_steps, decay_rate, staircase=True)
                            # Optimizer.
                            if momentum == 0:
                                optimizer = tf.train.GradientDescentOptimizer(learning_rate)
                            else: #applies momentum for increasing the robustness of the gradient 
                                optimizer = tf.train.MomentumOptimizer(learning_rate, momentum)
                            #grads = optimizer.compute_gradients(self.loss)
                            tvars = tf.trainable_variables()
                            #grads, _ = tf.clip_by_global_norm(tf.gradients(self.loss, tvars), clip_norm)
                            variable_names = [v.name for v in tvars]
                            variables_to_restore = {v.name.split(":")[0]: v for v in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)}
                            skip_pretrained_var = []
                            variables_to_restore = {v: variables_to_restore[v] for v in variables_to_restore if not any(x in v for x in skip_pretrained_var)}
                            if variables_to_restore:
                                saver_pre_trained = tf.train.Saver(var_list=variables_to_restore)
                                ckpt = tf.train.get_checkpoint_state('/content/gdrive/My Drive/MNIST_CayleyNet/model_save/')
                                saver_pre_trained.restore(self.session, ckpt.model_checkpoint_path) 
                            print('Relodaing parameters')
                            train_vars = [v for v in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES) if any(x in v.name for x in skip_pretrained_var)]

                            grads, variables = zip(*optimizer.compute_gradients(self.loss, var_list=tvars))
                            grads, _ = tf.clip_by_global_norm(grads, clip_norm)
                            self.op_gradients = optimizer.apply_gradients(zip(grads, variables), 
                                                                          global_step=global_step)
                            
                        #Computation of the norm gradients (useful for debugging)
                        self.var_grad = tf.gradients(self.loss, tf.trainable_variables())
                        self.norm_grad = self.frobenius_norm(tf.concat([tf.reshape(g, [-1]) for g in self.var_grad], 0))

                        #Extraction of the predictions and computation of accuracy
                        self.predictions = tf.cast(tf.argmax(self.logits, dimension=1), tf.int32)
                        self.accuracy = 100 * tf.contrib.metrics.accuracy(self.predictions, self.ph_labels)
        
                        uninitialized_vars = []
                        for var in tf.all_variables():
                            try:
                                self.session.run(var)
                            except tf.errors.FailedPreconditionError:
                                uninitialized_vars.append(var)

                        init_new_vars_op = tf.initialize_variables(uninitialized_vars)
                        self.session.run(init_new_vars_op)
                        self.count_no_weights()

In [0]:
#Convolutional parameters
p = [pool_step, pool_step, new_dim//(pool_step**2)]   #Dimensions of the pooling layers
K = [9, 9, 9] #List of gaussians distribution
F = [32, 32, 64] #Number of features of convolutional layers

#FC parameters
C = max(train_labels) + 1 # Number of classes we have
M = [512, C] # Number of neurons in fully connected layers

#Solver parameters
batch_size = 100
decay_steps = train_data.shape[0] / batch_size # number of steps to do before decreasing the learning rate
decay_rate = 0.95
learning_rate = 0.01
momentum = 0.9
regularization = 5e-4

# Definition of keep probabilities for dropout layers
dropout_training = 0.5
dropout_val_test = 1.0

num_jacobi_iter = 10

In [11]:
# Construction of the learning obj
M_0 = L[0].shape[0] # number of elements in the first graph
learning_obj = CayleyNet(p, K, F, M, M_0, batch_size, num_jacobi_iter, L_r, L_i,
                         decay_steps, decay_rate,
                         learning_rate=learning_rate, regularization=regularization,
                         momentum=momentum)#, clip_norm=100)


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
INFO:tensorflow:Restoring parameters from /content/gdrive/My Drive/MNIST_CayleyNet/model_save/model.ckpt-16501
Relodaing parameters
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use the `axis` argument instead
Instructions for updating:
Please use tf.global_variables instead.
Instructions for updating:
Use `tf.variables_initializer` instead.
#weights in the model: 100874


In [12]:
#Validation Code
tic = time.time()
val_accuracy = 0
for begin in range(0, val_data.shape[0], batch_size):
    end = begin + batch_size
    end = min([end, val_data.shape[0]])

    #data extraction
    batch_data = np.zeros((end-begin, val_data.shape[1]))
    batch_data = val_data[begin:end,:]
    batch_labels = np.zeros(batch_size)
    batch_labels[:end-begin] = val_labels[begin:end]

    feed_dict = {learning_obj.ph_data: batch_data, 
                  learning_obj.ph_labels: batch_labels,
                  learning_obj.ph_dropout: dropout_val_test}

    batch_accuracy = learning_obj.session.run(learning_obj.accuracy, feed_dict)
    val_accuracy += batch_accuracy*batch_data.shape[0]
val_accuracy = val_accuracy/val_data.shape[0]

val_time = time.time() - tic
msg = "[VAL] acc = %4.2f (%3.2es)" % (val_accuracy, val_time)
print(msg)

[VAL] acc = 33.80 (1.96e+00s)
